In [1]:
import os
from pprint import pprint
from groq import Groq
from openai import OpenAI
import anthropic
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if groq_api_key:
    print(f"groq API Key exists and begins {groq_api_key[:7]}")
else:
    print("Groq API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
groq API Key exists and begins gsk_p3V


In [2]:
LLM = "OPENAI"

In [3]:
if LLM == "OPENAI":
    client = OpenAI()
    MODEL = "gpt-4o-mini"
else:
    client = Groq()
    MODEL = "llama3-70b-8192"

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Extract content between two XML tags <tool>Tool description</tool>",
    }
)

Let's generate the first version of the essay.

In [6]:
mergesort_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)



generation_chat_history.append({"role": "assistant", "content": mergesort_code})

In [7]:
display_markdown(mergesort_code, raw=True)

To extract content between two XML tags in Python, you can use the `xml.etree.ElementTree` module which is part of the standard library. Here's a simple example that demonstrates how to extract the content between `<tool>` tags.

### Example Code

```python
import xml.etree.ElementTree as ET

def extract_tool_description(xml_string):
    # Parse the XML from the string
    root = ET.fromstring(xml_string)
    
    # Find the tool element and extract its text
    tool_element = root.find('tool')
    if tool_element is not None:
        return tool_element.text
    else:
        return None

# Example XML input
xml_data = """<root>
    <tool>Tool description</tool>
</root>"""

# Extract the tool description
description = extract_tool_description(xml_data)
print(description)  # Output: Tool description
```

### Explanation
1. **Import the Module**: We're importing `xml.etree.ElementTree` for handling XML.
2. **Function Definition**: We defined a function `extract_tool_description` that takes an XML string as input.
3. **Parsing XML**: The `ET.fromstring()` function parses the XML string into an element structure.
4. **Finding the Element**: The `find('tool')` method retrieves the first `<tool>` element found in the XML.
5. **Extracting Text**: Finally, the text within the `<tool>` tags is returned. If the `<tool>` element is not found, `None` is returned.
6. **Example XML**: We provided a sample XML string to demonstrate the function's usage.

Feel free to ask for more enhancements or examples!

In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [9]:
reflection_chat_history.append({"role": "user", "content": mergesort_code})

Now, let's generate a critique to the Python code.

In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
display_markdown(critique, raw=True)

Your example code for extracting content between XML tags using the `xml.etree.ElementTree` module is clear and well-structured. It demonstrates the necessary steps to parse an XML string and retrieve the text content of a specific tag quite effectively. Here are some critiques and recommendations for enhancing your code:

### Critique

1. **Error Handling**: Your code assumes that the XML is well-formed and does not account for possible parsing errors. If invalid XML is provided, `ET.fromstring()` will raise an error.

2. **Tag Hierarchy Consideration**: The current implementation only searches for the first `<tool>` tag directly under the root. If there are nested `<tool>` tags, this may not retrieve all instances, or it might not retrieve one from a deeper level if that’s the intended functionality.

3. **Text Content**: The code retrieves the text directly but does not handle cases where the `<tool>` element may contain nested elements. The text method would return only the direct text within the `<tool>` and ignore any sub-elements.

4. **Return Type**: Returning `None` for a missing tag is good practice; however, it might be useful to raise a custom exception or to return a default value to signal that no content was found more explicitly.

### Recommendations

1. **Add Error Handling**:
   Use try-except blocks to handle possible XML parsing errors gracefully.

   ```python
   try:
       root = ET.fromstring(xml_string)
   except ET.ParseError as e:
       print(f"Error parsing XML: {e}")
       return None
   ```

2. **Search for All Tags**:
   If you want to retrieve all `<tool>` elements rather than just the first one, you could use `findall()` instead.

   ```python
   tool_elements = root.findall('tool')
   return [tool.text for tool in tool_elements if tool.text is not None]
   ```

3. **Handling Nested Elements**:
   To retrieve text from nested elements, consider using `ElementTree`'s `itertext()` method. 

   ```python
   def extract_tool_description(xml_string):
       ...
       tool_element = root.find('tool')
       return ''.join(tool_element.itertext()) if tool_element is not None else None
   ```

4. **Improve Documentation**:
   Adding more detailed comments and possibly docstrings to explain function parameters, return types, and exceptions can help make the code easier to use and maintain.

5. **Enhance Flexibility**:
   Consider allowing the function to accept parameters for the tag name, making it more reusable for different XML structures.

   ```python
   def extract_tag_description(xml_string, tag_name='tool'):
       ...
       tag_element = root.find(tag_name)
       ...
   ```

Here’s an enhanced version of your code that incorporates some of these suggestions:

```python
import xml.etree.ElementTree as ET

def extract_tool_description(xml_string, tag_name='tool'):
    """Extracts the text content of specified XML tags.

    Args:
        xml_string (str): A string containing XML data.
        tag_name (str): The tag name to search for. Default is 'tool'.

    Returns:
        str: The text content of the found tag, or None if not found.
    """
    try:
        root = ET.fromstring(xml_string)
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None

    tag_element = root.find(tag_name)
    if tag_element is not None:
        return ''.join(tag_element.itertext())
    else:
        return None

# Example XML input
xml_data = """<root>
    <tool>Tool description with nested <feature>Feature A</feature> details.</tool>
</root>"""

# Extract the tool description
description = extract_tool_description(xml_data)
print(description)  # Output: Tool description with nested Feature A details.
```

These changes improve the robustness, flexibility, and usability of your code while maintaining clarity.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [12]:
generation_chat_history.append({"role": "user", "content": critique})

## Generation Step (II)

In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
display_markdown(essay, raw=True)

Thank you for your detailed critique and recommendations! They are very helpful in improving the code. In response to your suggestions, I've made some enhancements to the previous implementation, focusing on error handling, flexibility, and functionality to handle nested XML elements. 

Here's the revised version:

### Enhanced Version of the XML Content Extraction Code

```python
import xml.etree.ElementTree as ET

def extract_tag_description(xml_string, tag_name='tool'):
    """Extracts the text content of specified XML tags, including nested tags.

    Args:
        xml_string (str): A string containing XML data.
        tag_name (str): The tag name to search for. Default is 'tool'.

    Returns:
        str or None: The combined text content of the found tag(s), or None if not found or on error.
    """
    try:
        # Parse the XML from the string
        root = ET.fromstring(xml_string)
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None

    # Find all elements with the specified tag name
    tag_elements = root.findall(tag_name)
    if tag_elements:
        # Return combined text from all found tags
        return ' '.join(''.join(tag.itertext()).strip() for tag in tag_elements)
    else:
        return None

# Example XML input
xml_data = """<root>
    <tool>Tool description with nested <feature>Feature A</feature> details.</tool>
    <tool>Another tool description.</tool>
</root>"""

# Extract descriptions of all tool elements
descriptions = extract_tag_description(xml_data)
print(descriptions)  # Output: Tool description with nested Feature A details. Another tool description.
```

### Key Enhancements:
1. **Error Handling**: The code now includes a try-except block to gracefully handle parsing errors.
2. **Multiple Tag Retrieval**: The function uses `findall()` to extract all occurrences of the specified tags.
3. **Nested Content Handling**: The use of `itertext()` ensures that all text, including from nested elements, is extracted.
4. **Function Documentation**: Added a detailed docstring explaining parameters and the return type.
5. **Flexibility**: The function allows for the tag name to be specified as an argument, making it reusable for different XML structures.

This enhanced version should adequately meet your critiques and offers improved usability and reliability. If there are any additional features you would like to see or further improvements, please let me know!